In [6]:
import matplotlib.pyplot as plt  
import numpy as np
import os                        
from tensorflow import keras
from keras.models import load_model
import tensorflow as tf
import random

In [7]:

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
   rescale=1./255,
   shear_range = 0.15,
   zoom_range = 0.15,
   horizontal_flip = True
)

img_height=256
img_width=256
bs=32

train_ds = datagen.flow_from_directory(
    'split_dataset/train',
    target_size = (img_height, img_width),
    batch_size = bs,
    class_mode='binary',
    shuffle=True)

val_ds = datagen.flow_from_directory(
    'split_dataset/val',
    target_size = (img_height, img_width),
    batch_size = bs,
    class_mode='binary',
    shuffle=False)

test_ds = datagen.flow_from_directory(
    'split_dataset/test',
    target_size = (img_height, img_width),
    batch_size = bs,
    class_mode='binary',
    shuffle=False)

Found 3820 images belonging to 2 classes.
Found 474 images belonging to 2 classes.
Found 488 images belonging to 2 classes.


In [8]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense,Dropout
from keras.layers import BatchNormalization
from keras.layers import Input

input_shape = (256, 256, 3)

classifier = Sequential()

classifier.add(Input(shape=input_shape))

classifier.add(Conv2D(32, (3, 3), activation = 'relu'))

classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Flatten())

classifier.add(Dense(units = 512, activation = 'relu'))
classifier.add(BatchNormalization()),
classifier.add(Dense(256,activation='relu')),
classifier.add(Dropout(0.25)),
classifier.add(Dense(units = 1, activation = 'sigmoid'))


classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [9]:
classifier.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 254, 254, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 125, 125, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 62, 62, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 123008)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 512)                 │      62,980,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 63,124,385 (240.80 MB)

 Trainable params: 63,123,361 (240.80 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [10]:
total_train_samples = 1910 + 1910  
total_val_samples = 237 + 237

spe = int(round(total_train_samples/32))      #32 is batch size
vs = int(round(total_val_samples/32))

model_info = classifier.fit(
    train_ds,
    steps_per_epoch = spe,
    epochs = 211,
    validation_data = val_ds,
    validation_steps = vs
)

Epoch 1/211
119/119 ━━━━━━━━━━━━━━━━━━━━ 186s 2s/step - accuracy: 0.6433 - loss: 0.6846 - val_accuracy: 0.7890 - val_loss: 0.4970
Epoch 2/211
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 818us/step - accuracy: 0.7500 - loss: 0.2380 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/211


C:\Program Files\Python312\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


119/119 ━━━━━━━━━━━━━━━━━━━━ 176s 1s/step - accuracy: 0.7761 - loss: 0.4830 - val_accuracy: 0.7595 - val_loss: 0.5641
Epoch 4/211
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 165us/step - accuracy: 0.7188 - loss: 0.3023 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/211
119/119 ━━━━━━━━━━━━━━━━━━━━ 177s 1s/step - accuracy: 0.7901 - loss: 0.4415 - val_accuracy: 0.7658 - val_loss: 0.5081
Epoch 6/211
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 180us/step - accuracy: 0.9062 - loss: 0.1444 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/211
119/119 ━━━━━━━━━━━━━━━━━━━━ 176s 1s/step - accuracy: 0.8183 - loss: 0.4172 - val_accuracy: 0.6456 - val_loss: 0.7761
Epoch 8/211
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 255us/step - accuracy: 0.8438 - loss: 0.1985 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/211
119/119 ━━━━━━━━━━━━━━━━━━━━ 175s 1s/step - accuracy: 0.8175 - loss: 0.3952 - val_accuracy: 0.5464 - val_loss: 1.0435
Epoch 10/211
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 386us/step - accuracy: 0.7500 

In [12]:
score=classifier.evaluate(test_ds)
print("Loss:",score[0],"Accuracy:",score[1])

16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 500ms/step - accuracy: 0.8982 - loss: 0.3028
Loss: 0.3834225535392761 Accuracy: 0.9036885499954224


In [13]:
classifier.save('glaucoma_detection1.h5')